# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://www.scripters.academy/")
print(ed.get_contents())

Webpage Title:
الرئيسية
Webpage Contents:





## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.scripters.academy/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):



In [9]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'},
  {'type': 'api page', 'url': 'https://anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new 

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}


# Anthropic Brochure

---

## **Company Overview**

**Anthropic** is a pioneering AI safety and research company headquartered in San Francisco. Our mission is centered around building AI systems that are reliable, interpretable, and steerable, while ensuring their safe integration into everyday life. With our state-of-the-art AI model, **Claude**, we provide tools designed to empower organizations while maintaining a strong emphasis on safety.

---

## **Our Purpose**

We recognize the transformative potential of AI on a global scale and strive to develop systems that individuals and organizations can trust. Through rigorous research and collaborative efforts, we aim to navigate the complex landscape of AI opportunities and risks.

---

## **Safety is a Science**

At Anthropic, AI safety is treated as a systematic science. This involves conducting comprehensive research, actively applying insights into our products, and sharing our findings within the industry and with the public.

### **Interdisciplinary Team**

Our team comprises experts from various fields including machine learning, physics, policy, and business operations, all integrated to develop beneficial AI systems. 

---

## **Our Products**

### **Claude**
Meet **Claude 3.5 Sonnet**, our most intelligent AI model, enabling teams to enhance efficiency and drive innovation.

- **Enterprise Solutions**: Securely connect Claude to your company knowledge, empowering every team with trusted AI.
- **API Access**: Integrate Claude into your applications to leverage advanced AI capabilities.

---

## **Customer Focus**

Our customers range from businesses to nonprofits, all benefiting from Claude's advanced capabilities. We prioritize collaboration with civil society, government entities, and industry stakeholders to ensure safe and effective AI utilization.

---

## **Company Culture**

At Anthropic, collaboration and trust form the foundation of our culture. Here are some of our core values:

1. **Mission-Driven**: We exist to ensure AI benefits people and society.
2. **High Trust**: We promote honesty, emotional maturity, and open disagreement.
3. **One Big Team**: Collaboration is central, allowing us to pursue our goals collectively.
4. **Simplicity & Pragmatism**: We value straightforward solutions and empiricism in all aspects of our work.

---

## **Careers at Anthropic**

Join us in building the future of safe AI! We celebrate diverse backgrounds and rich experiences, understanding that innovative solutions arise from a blend of perspectives.

### **What We Offer:**

- **Health & Wellness**: Comprehensive insurance, mental health support, and flexible time-off policies.
- **Competitive Compensation**: Salary and equity packages along with generous retirement plans.
- **Additional Benefits**: Culture of learning with stipends for wellness, education, and home office setups.

### **The Hiring Process**

We aim to minimize bias through a structured interview process that includes exploratory chats, skills assessments, team screens, and final checks to find the best fit for our multidisciplinary team.

### **Work Environment**

While we are headquartered in the Bay Area, we promote flexible work arrangements, often allowing remote work options and supporting those who relocate.

---

## **Join Us**

Are you ready to be part of a forward-thinking company shaping the future of AI safely? Visit our [Careers page](#) to see open positions!

---

## **Get in Touch**

For more information, check our website or follow us on social media:
- [Twitter](#)
- [LinkedIn](#)
- [YouTube](#)

Together, let's build reliable and beneficial AI systems. 

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic: Pioneering Safe and Reliable AI

## Welcome to Anthropic

Anthropic is a San Francisco-based AI safety and research company committed to building reliable and interpretable AI systems that prioritize safety through innovative research and thoughtful design. With a mission centered around ensuring that transformative AI technologies benefit society, we are focused on empowering our customers through our state-of-the-art AI model, Claude.

---

## Our Core Beliefs

- **Safety is a Science**: We approach AI safety as a systematic discipline, conducting rigorous research and applying findings to our products.
- **Interdisciplinary Collaboration**: Our team of experts spans disciplines including machine learning, policy, physics, and business, working together toward shared goals.
- **Community Engagement**: We actively collaborate with civil society, government, and academia to promote AI safety industry-wide.

---

## Meet Claude

Claude is our flagship AI assistant, designed to enhance productivity and creativity across teams. Claude not only streamlines daily tasks but also serves as a collaborative partner, boosting innovation and decision-making through shared knowledge and insights.

### Key Features:
- **Easy Collaboration**: Claude helps teams work smarter, generating ideas and performing complex tasks more efficiently.
- **Data Protection**: Customer confidentiality is paramount, and all data remains secure—we do not train our models on your sensitive information.
- **Enterprise Integration**: Empower organizations to scale their internal expertise and knowledge with Claude effortlessly.

---

## Company Culture

At Anthropic, we cultivate an environment of **high trust**, collaboration, and **pragmatism**. Our core values emphasize working together towards our mission, assuming good intentions, and pursuing excellence through simple, effective solutions.

### Our Values Include:
- **Mission-Driven**: We're here to ensure AI supports human flourishing.
- **Collaborative Spirit**: We function as one big team where every voice matters in shaping our path forward.
- **Openness and Honesty**: We encourage emotional maturity and intellectual openness at all levels.

---

## Join Us

Anthropic is a **Public Benefit Corporation** dedicated to the long-term well-being of humanity through responsible AI. We are seeking talented individuals from diverse backgrounds to join our mission of building safer AI systems.

### What We Offer:
- **Health & Wellness**: Comprehensive benefits, including health, dental, and vision insurance, parental leave, and mental health support.
- **Compensation**: Competitive salaries, equity packages, and retirement plans.
- **Professional Development**: Flexible stipends for wellness, education, and home office setups.

### Career Opportunities
We are always on the lookout for individuals talented in various fields such as engineering, policy, and operations. Our recruitment process values unique strengths and diverse experiences.

---

## Customer Commitment

Our customers range from enterprises and nonprofits to educational institutions seeking to leverage AI for innovative applications. With Claude, businesses enhance operations, foster collaboration, and drive growth, all while ensuring safety and reliability.

### Join the teams transforming their operations with Claude's AI capabilities!

---

For more information on our products, career opportunities, or to explore our cutting-edge research, visit our [website](https://www.anthropic.com). Together, let’s shape the future of AI!

---

**Connect with Us:**
- [Twitter](https://twitter.com/anthropicai)
- [LinkedIn](https://www.linkedin.com/company/anthropic/)
- [YouTube](https://www.youtube.com/channel/UCxZgY7lzOs3AEs9H5HKA4bg)

© 2024 Anthropic PBC. All rights reserved.

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

## Business Applications

In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.